In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np 
import matplotlib.pyplot as plt
from IPython import display
import glob
import scipy

from src.mat_dataset import MAT_Dataset
from torch.utils.data import DataLoader
from torchsummary import summary

from src.metrics import complex_MSE


In [2]:
file_list = glob.glob('./data/*')
train_dataset = MAT_Dataset(file_list[0], [0,1,2])
test_dataset = MAT_Dataset(file_list[0], [3])

batch_size = 2
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)


In [9]:

#===============================

In [4]:
from src.models import Beam_DnCNN_3D
from src.preprocessing import AntFreq2BeamTime_3D, BeamDelay2AntFreq_3D
from src.preprocessing import Pol2Dim, PowerNoiseNormalizer

In [ ]:
model = Beam_DnCNN_3D(cfg = cfg)

In [26]:
x = next(iter(train_loader)).type(torch.complex64)

y = model.WindowCut(
    model.complex_to_dim(
    model.left_transform(x, norma = torch.tensor([0.1, 0.1]))
    ))

x_ = model.right_transform(
     model.dim_to_complex(
     model.WindowCut(y, 'backward')))

print(torch.sum(abs(x - x_)**2))
print(torch.sum(abs(x)**2))

tensor(7.8891e-09)
tensor(313344.)


In [28]:
x_denoised, noise_pred, L1 = model(x, torch.tensor([0.1, 0.1]))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#=======================================================================
cfg = {}
cfg['N_hor'] = 8
cfg['N_ver'] = 4
cfg['N_pol'] = 2
cfg['N_subc'] = 612 
cfg['N_UE_ANT'] = 4 


cfg['N_Az'] = 64
cfg['N_El'] = 32
cfg['N_time'] = 1024 
cfg['window_cut'] = (32,16,70)
#=======================================================================

model = Beam_DnCNN_3D(cfg = cfg, n_layers = 15, n_features = 20)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [150, 200, 250], gamma = 0.1)

train_model(model = model,
            epochs = 200,
            trainLoader = train_loader,
            testLoader = test_loader,
            optimizer = optimizer,
            scheduler = scheduler,
            criterion = complex_MSE)